In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import KarateClub, Planetoid, FacebookPagePage
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from sklearn.metrics import normalized_mutual_info_score, f1_score, accuracy_score
import networkx as nx
from torch_geometric.utils import to_networkx
import numpy as np


class GCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = F.relu(x)
        return x

class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNLayer, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        x, (h_n, c_n) = self.rnn(x)
        return x, (h_n, c_n)

class HybridModel(nn.Module):
    def __init__(self, gcn_input_dim, gcn_output_dim, rnn_input_dim, rnn_hidden_dim, rnn_num_layers, num_classes):
        super(HybridModel, self).__init__()
        self.gcn = GCNLayer(gcn_input_dim, gcn_output_dim)
        self.rnn = RNNLayer(rnn_input_dim, rnn_hidden_dim, rnn_num_layers)
        self.fc = nn.Linear(rnn_hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # GCN forward
        x = self.gcn(x, edge_index)
        x = x.unsqueeze(1)

        # RNN forward
        x, (h_n, c_n) = self.rnn(x)


        x = self.fc(x[:, -1, :])
        return F.log_softmax(x, dim=-1)


def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()


def test(model, data):
    model.eval()
    logits, accs = model(data), []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        logits = model(data)
        preds = logits.max(1)[1].cpu().numpy()
        labels = data.y.cpu().numpy()


    accuracy = accuracy_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    f1 = f1_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()], average='macro')


    nmi = normalized_mutual_info_score(labels[data.test_mask.cpu()], preds[data.test_mask.cpu()])


    G = to_networkx(data)
    communities = {i: preds[i] for i in range(len(preds))}
    community_list = [[] for _ in range(data.num_classes)]
    for node, community in communities.items():
        community_list[community].append(node)
    modularity = nx.algorithms.community.modularity(G, community_list)

    return accuracy, f1, nmi, modularity


def load_data(dataset_name):
    if dataset_name == 'KarateClub':
        data = KarateClub(transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    elif dataset_name == 'FacebookPagePage':
        data = FacebookPagePage(root='/tmp/FacebookPagePage', transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))
    else:
        data = Planetoid(root=f'/tmp/{dataset_name}', name=dataset_name, transform=NormalizeFeatures())[0]
        data.num_classes = len(set(data.y.tolist()))


    num_nodes = data.y.size(0)
    indices = np.random.permutation(num_nodes)
    train_size = int(num_nodes * 0.6)
    val_size = int(num_nodes * 0.2)
    test_size = num_nodes - train_size - val_size

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    return data


datasets = ['KarateClub', 'Cora', 'Citeseer', 'FacebookPagePage']
for dataset_name in datasets:
    data = load_data(dataset_name)


    model = HybridModel(gcn_input_dim=data.num_node_features, gcn_output_dim=32,
                        rnn_input_dim=32, rnn_hidden_dim=64, rnn_num_layers=2, num_classes=data.num_classes)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()


    for epoch in range(180):
        loss = train(model, data, optimizer, criterion)
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')


    accuracy, f1, nmi, modularity = evaluate(model, data)
    print(f'{dataset_name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, NMI: {nmi:.4f}, Modularity: {modularity:.4f}')


Epoch 0, Loss: 1.4025779962539673
Epoch 10, Loss: 1.1635318994522095
Epoch 20, Loss: 0.4141424596309662
Epoch 30, Loss: 0.0777798444032669
Epoch 40, Loss: 0.0005754625890403986
Epoch 50, Loss: 0.00011458154767751694
Epoch 60, Loss: 8.227933722082525e-05
Epoch 70, Loss: 6.816041423007846e-05
Epoch 80, Loss: 5.8821278798859566e-05
Epoch 90, Loss: 5.13713966938667e-05
Epoch 100, Loss: 4.569748853100464e-05
Epoch 110, Loss: 4.138837175560184e-05
Epoch 120, Loss: 3.809242480201647e-05
Epoch 130, Loss: 3.5475917684379965e-05
Epoch 140, Loss: 3.331236803205684e-05
Epoch 150, Loss: 3.1482581107411534e-05
Epoch 160, Loss: 2.9891194571973756e-05
Epoch 170, Loss: 2.8496497179730795e-05
KarateClub - Accuracy: 0.8750, F1 Score: 0.7000, NMI: 0.9007, Modularity: 0.3898
Epoch 0, Loss: 1.9356201887130737
Epoch 10, Loss: 1.816191554069519
Epoch 20, Loss: 1.6685070991516113
Epoch 30, Loss: 1.2845604419708252
Epoch 40, Loss: 0.9115021824836731
Epoch 50, Loss: 0.5922475457191467
Epoch 60, Loss: 0.415209352